Responsabilidades:
- Definir las especificaciones de desempeño: %OS y Ts.
- Calcular ζ y ωn, luego los polos dominantes deseados.
- Resolver las condiciones de fase y módulo para obtener Kp, Ti, Td, con ayuda simbólica o numérica.
- Ajustar si es necesario para cumplir condiciones.
<!--  -->
Entregables:
- Cálculos desarrollados (puede usar SymPy o MATLAB Symbolic Toolbox).
- Tabla con valores obtenidos y justificación
- Validación de que el punto cumple ∠L = -180° y |L| = 1


### Desarrollo
- Sobreimpulso (%OS): mide cuánto supera temporalmente la salida su valor final al aplicar una entrada escalón

Se calcula así: %OS = $\frac{Y_{max} - Y_{final}}{Y_{final}}$

**Para el Caso actual debera ser de 10%**

- Tiempo de asentamiento (Ts): es cuánto tarda la respuesta en mantenerse dentro de un rango pequeño

**Para el Caso actual debera ser 2 minutos**

### Encontrar $\zeta $ y $w_n$

La ecuación para encontrar $\zeta$ y $\omega_n$ es:

%OS $  \approx 100, e^{-\frac{\pi \zeta}{\sqrt{1-\zeta^2}}} $

$ T_s \approx \frac{4}{\zeta \omega_n}, \quad \omega_n = \text{frecuencia natural}. $


In [6]:
from sympy import symbols, Eq, exp, sqrt, pi, solve, N

zeta, wn, Ts, OS = symbols('zeta wn Ts OS', real=True, positive=True)

OS_value = 10  # 10%
Ts_value = 2   # 2 minutos

# Ecuación del sobreimpulso (OS ≈ 100 * exp(-pi*zeta / sqrt(1 - zeta**2)))
eq1 = Eq(OS, 100 * exp(-pi * zeta / sqrt(1 - zeta**2)))
# Ecuación del tiempo de asentamiento (Ts ≈ 4 / (zeta * wn))
eq2 = Eq(Ts, 4 / (zeta * wn))
# Sustituir valores numéricos
eq1_num = eq1.subs(OS, OS_value)
eq2_num = eq2.subs(Ts, Ts_value)

# Resolver la ecuación de OS para zeta
zeta_solutions = solve(eq1_num, zeta)
# Evaluar las soluciones numéricamente  
zeta_numeric = [N(sol) for sol in zeta_solutions if sol.is_real and sol > 0]
# Usar el valor numérico de zeta para encontrar wn
wn_val = solve(eq2_num.subs(zeta, zeta_numeric[0]), wn)[0]
zeta= zeta_numeric[0]
print("Valor de zeta:", zeta)
print("Valor de wn:", wn_val)

Valor de zeta: 0.591155033798898
Valor de wn: 3.38320725639016


$ s_{1,2} = -\zeta \omega_n \pm j \omega_n \sqrt{1 - \zeta^2} $  

In [9]:
s1 = symbols('s1', real=True, positive=True)
# Ecuación del polo dominante
first_pole = -zeta * wn_val + wn_val * sqrt(1 - zeta**2)*1j
print("Polo S1:", first_pole)
s2 = symbols('s2', real=True, positive=True)
# Ecuación del segundo polo
second_pole = -zeta * wn_val - wn_val * sqrt(1 - zeta**2)*1j
print("Polo S2:", second_pole)

Polo S1: -2.0 + 2.72875270768368*I
Polo S2: -2.0 - 2.72875270768368*I


### Condiciones de fase y modulo
- Fase

$\angle (T_i T_d s_d^2 + T_i s_d + 1) - \angle s_d - \angle (\tau s_d + 1) = -180^\circ - \angle K_p - \angle K.$

- Modulo

$K_p K \frac{\left| T_i T_d s_d^2 + T_i s_d + 1 \right|}{\left| T_i \right| \left| s_d \right| \left| \tau s_d + 1 \right|} = 1.$

In [1]:
from sympy import symbols, I, arg, simplify, Eq, pi, Abs

# Definir variables simbólicas
Ti, Td, tau = symbols('Ti Td tau', real=True, positive=True)
Kp = symbols('Kp', real=True, positive=True)
# Parámetros de la planta (asumidos si no hay datos reales)
K = 1  # ganancia
tau_val = 1  # constante de tiempo de la planta


# Polo deseado (uno de ellos)
sd = -2.0 + 2.72875270768368 * I

# Numerador simbólico: Ti*Td*s^2 + Ti*s + 1
numerador = Ti * Td * sd**2 + Ti * sd + 1


# Denominadores individuales
den_sd = sd
den_tau = tau * sd + 1
numerador_mod = Abs(numerador)
den_sd_mod = Abs(sd)
den_tau_mod = Abs(den_tau.subs(tau, tau_val))

# Ecuación de fase: angle(numerador) - angle(sd) - angle(tau*sd + 1) = -pi
fase_eq = Eq(arg(numerador) - arg(den_sd) - arg(den_tau.subs(tau, tau_val)), -pi)
modulo_eq = Eq((Kp * K * numerador_mod) / (Ti * den_sd_mod * den_tau_mod), 1)

# Simplificar la ecuación
print(fase_eq)
print(modulo_eq)



Eq(arg(Td*Ti*(-2.0 + 2.72875270768368*I)**2 + Ti*(-2.0 + 2.72875270768368*I) + 1) - 2*pi + 2.15783302647891, -pi)
Eq(0.101705282274658*Kp*sqrt(Td**2*Ti**2*(2.0 - 2.72875270768368*I)**2*(2.0 + 2.72875270768368*I)**2 - 2.0*Td*Ti**2*(2.0 + 2.72875270768368*I)**2 + 2.72875270768368*I*Td*Ti**2*(2.0 + 2.72875270768368*I)**2 - 2.72875270768368*I*Td*Ti**2*(2.0 - 2.72875270768368*I)**2 - 2.0*Td*Ti**2*(2.0 - 2.72875270768368*I)**2 + Td*Ti*(2.0 - 2.72875270768368*I)**2 + Td*Ti*(2.0 + 2.72875270768368*I)**2 + 11.446091339691*Ti**2 - 4.0*Ti + 1)/Ti, 1)
